In [1]:
from pytfa.io.json import load_json_model
model = load_json_model('./../models/TFA_GEM_Recon3_Lehninger_Curated_prepared.json')

2025-01-27 15:09:45,217 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [2]:
# This is the models with the original annotation 
# TODO: Put the original annotation back into the TFA 
# (its lost becuse the TFA model was made in matTFA ... )
from cobra.io.json import load_json_model
recon3 = load_json_model('./../models/Recon3D.json')

In [3]:
# Add a defined medium 

medium_metabolites = {
    'EX_o2_e': -100,
    'EX_h2o_e': -100,
    'EX_pi_e': -100,
    'EX_nh4_e': -100,
    'EX_h_e': -100,
    'EX_so4_e': -100,
    'EX_hco3_e': -100,
    'EX_h2o2_e': -100,
    'EX_glc_D_e': -100,
    'EX_ac_e': -100,
    'EX_lac_L_e': -100,
    'EX_hdca_e': -100,
    'EX_lnlc_e': -100,
    'EX_ocdcea_e': -100,
    'EX_ala_L_e': -100,
    'EX_arg_L_e': -100,
    'EX_asn_L_e': -100,
    'EX_asp_L_e': -100,
    'EX_cys_L_e': -100,
    'EX_gln_L_e': -100,
    'EX_glu_L_e': -100,
    'EX_gly_e': -100,
    'EX_his_L_e': -100,
    'EX_ile_L_e': -100,
    'EX_leu_L_e': -100,
    'EX_lys_L_e': -100,
    'EX_met_L_e': -100,
    'EX_phe_L_e': -100,
    'EX_pro_L_e': -100,
    'EX_ser_L_e': -100,
    'EX_thr_L_e': -100,
    'EX_trp_L_e': -100,
    'EX_tyr_L_e': -100,
    'EX_val_L_e': -100,
    'EX_bhb_e': -100,
    'EX_chol_e': -100,
}

for m, b in medium_metabolites.items():
    model.reactions.get_by_id(m).lower_bound = b

# Also allow for cytostolic ATP to be produced
model.reactions.cyt_atp2adp.lower_bound = -100

In [4]:
# Read the met_label table 
import pandas as pd

met_labels = pd.read_excel('Met_labeling_metabolites.xlsx')

In [16]:
# from genome-scale metabolic models

def find_metabolites_from_annotation(id, model):
    """
    Find metabolites in a model based on a given annotation id

    Parameters
    ----------
    id : str
        The annotation id to search for
    model : cobra.Model
        The model to search in  
        
    """

    # Veryfi if id is a non empty string
    if id and isinstance(id, str):
        metabolites = [] 
        for met in model.metabolites:
            for id_type, ids in met.annotation.items():
                # TODO fix HMDB old vs new
                if id_type == 'hmdb':
                    # Compare only last 5 digits of HMDB ids
                    if id[-5:] in [i[-5:] for i in ids]:
                        metabolites.append(met.id)
                if id in ids:
                    metabolites.append(met.id)
        if metabolites not in [[], None]:            
            return metabolites
        else:
            return None
    else:
        return None

In [17]:
#from metabolink.annotation import find_metabolites_from_annotation

# Try to dinf the metablie labels using KEGG and HMDB 
kegg_2_met = met_labels.apply(lambda x: find_metabolites_from_annotation(x['KEGG'], recon3), axis=1)
hmdb_2_met = met_labels.apply(lambda x: find_metabolites_from_annotation(x['HMDB'], recon3), axis=1)


In [18]:
hmdb_2_met

0                                              None
1                                [3sala_m, 3sala_c]
2                                  ta_c, 5mta_e]
3                             [ach_e, ach_n, ach_c]
4                                              None
5                                              None
6                                              None
7                       [creat_m, creat_c, creat_e]
8                                              None
9              [gthrd_c, gthrd_e, gthrd_r, gthrd_m]
10                                             None
11                                             None
12         [acrn_c, acrn_m, acrn_x, acrn_r, acrn_e]
13              [crn_c, crn_m, crn_x, crn_e, crn_r]
14                   [met__L_c, met__L_e, met__L_m]
15                                             None
16                [HC00900_m, HC00900_c, HC00900_e]
17                   [C02712_m, C02712_c, C02712_e]
18                                             None
19             

In [6]:
# Make a unified list of model metabolites 
metabolites = set()
for m in kegg_2_met:
    if m is not None:
        metabolites.update(m)
for m in hmdb_2_met:
    if m is not None:
        metabolites.update(m)
        
metabolites = list(metabolites)
metabolites.sort()

# Subs two underscores for one
metabolites = [m.replace('__', '_') for m in metabolites]

In [7]:
from metabolink import find_precursor_sets

# find precursor sets for all metabolites found in RECON3
precursor_sets = []
for m in metabolites:
    try:
        precursor_set = find_precursor_sets(metabolites, model)
        precursor_sets.append(precursor_set)
    except ValueError as e:
        print(f'Error for {m}: {e}')


Error for 3sala_c: Metabolites cannot be produced


Ignoring reaction 'pseudo_biomass' since it already exists.


AttributeError: 'NoneType' object has no attribute 'name'

In [ ]:
# Join the precursor list of dicts into a single dict 
precursor_dict = {}
for p in precursor_sets:
    precursor_dict.update(p)

In [19]:
from metabolink.io import extract_precursor_sets
table = extract_precursor_sets(precursor_dict)

{'seed_id': 'cpd00060'}